In [1]:
import numpy as np
import pandas as pd

import os
import random

import warnings

warnings.filterwarnings(action='ignore')

path = '/opt/ml/input/data/'
train = pd.read_csv(path + 'train_data.csv')
test = pd.read_csv(path + 'test_data.csv')

k = 100 # k개까지만 문제 기록을 남기려고함.

In [2]:
# 어짜피 시간순으로 정렬되어 있음. 시간 역순으로 정렬함.
dat = pd.concat([train, test])
dat = dat.loc[::-1]
dat.reset_index(drop= True, inplace= True)
#dat = dat.sort_values(by = ['userID', 'Timestamp'], ascending= False)
dat['tem'] = 1
dat['seq'] = dat.groupby('userID')['tem'].apply(lambda x : x.cumsum())

test_id = test['userID'].unique()
test = dat[(dat['userID'].isin(test_id)) & (dat['seq'] <= k)]
train = dat[~((dat['userID'].isin(test_id)) & (dat['seq'] == 1))]
train['seq'] = train.apply(lambda x : x['seq'] - 1  if x['userID'] in test_id else x['seq'], axis = 1)

user_num = dat['userID'].nunique()
valid_data = train[train['seq'] <= k]
train_new = train.copy()
train_new['seq'] = train_new['seq'] - 1
train_new = train_new[train_new['seq'] >= 1]

train_datas = train_new[train_new['seq'] <= k]
train_new = train_new[train_new['seq'] > k]
train_new['userID'] = train_new['userID'] + user_num

for i in range(3, 1860 // k + 2):
    train_data = train_new[train_new['seq'] <= k * i]
    train_new = train_new[train_new['seq'] > k * i]
    train_datas = pd.concat([train_datas, train_data])
    train_new['userID'] = train_new['userID'] + user_num

In [9]:
train

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,tem,seq
2526055,0,A060127007,A060000127,1,2020-10-20 05:20:05,713,1,100
2526054,0,A060129001,A060000129,1,2020-10-21 05:41:03,714,1,99
2526053,0,A060129002,A060000129,1,2020-10-21 05:41:13,714,1,98
2526052,0,A060129003,A060000129,1,2020-10-21 05:42:16,714,1,97
2526051,0,A060129004,A060000129,1,2020-10-21 05:43:34,714,1,96
...,...,...,...,...,...,...,...,...
2005895,127244,A040031001,A040000031,0,2020-02-27 04:01:38,2065,1,5
2005894,127244,A040031002,A040000031,0,2020-02-27 04:02:10,2065,1,4
2005893,127244,A040031003,A040000031,1,2020-02-27 04:02:28,2065,1,3
2005892,127244,A040031004,A040000031,1,2020-02-27 04:02:51,2065,1,2


In [6]:
test['userID'].nunique()

744

In [7]:
valid['userID'].nunique()

7420

In [8]:
train['userID'].nunique()

24242

In [5]:
train_datas['seq'] = (train_datas['seq']-1) % k + 1
valid_data['seq'] = (valid_data['seq']-1) % k + 1
test['seq'] = (test['seq']-1) % k + 1

train_datas.sort_values(['userID', 'seq'], ascending= [True,False], inplace= True)
valid_data.sort_values(['userID', 'seq'], ascending= [True,False], inplace= True)
test.sort_values(['userID', 'seq'], ascending= [True,False], inplace= True)

tem = valid_data.drop_duplicates(['userID'], keep='first')
lst = tem[tem['seq'] <= 14]['userID'].unique()
valid = valid_data[~(valid_data['userID'].isin(lst))]

tem = train_datas.drop_duplicates(['userID'], keep='first')
lst = tem[tem['seq'] <= 14]['userID'].unique()
train = train_datas[~(train_datas['userID'].isin(lst))]

train.drop(['tem'], axis = 1).to_csv(path + 'GKT/train_data2.csv', index = False)
valid.drop(['tem'], axis = 1).to_csv(path + 'GKT/valid_data2.csv', index = False)
test.drop(['tem'], axis = 1).to_csv(path + 'GKT/test_data2.csv', index = False)